In [12]:
import sys
sys.path.append('/home/nick/python/asop_global/ASoP-Coherence')
from asop_coherence_global_temporal import load_cmip6,mask_wet_season,mask_min_precip,new_cube_copy
import iris
from pathlib import Path
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import iris.coord_categorisation
from iris.experimental.equalise_cubes import equalise_attributes
from iris.util import unify_time_units
import dask
from dask.distributed import Client,progress

In [13]:
def haversine(origin, destination):
    import math

    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

In [14]:
def get_asop_dict(key):
    cmip6_path=Path('/media/nick/lacie_tb3/data_from_gill/CMIP6')
    obs_path=Path('/media/nick/lacie_tb3/datasets')
    if key == 'GPM_IMERG':
        asop_dict={
            'desc': '3B-HHR.MS.MRG.3IMERG.V06B.3hr_means_3x3',
            'dir': obs_path/'GPM_IMERG',
            'file_pattern': '3B-HHR.MS.MRG.3IMERG.*.3hr_means_3x3.V06B.nc',
            'name': 'IMERG-3B-V06',
            'start_year': 2001,
            'stop_year': 2018,
            'dt': 10800,
            'legend_name': 'IMERG',
            'region': [-60,60,0,360],
            'color': 'black',
            'symbol': '>',
            'region_size': 3,
            'lag_length': 2,
            'dx': 330,
            'dy': 330
        }
    else:
        raise Exception('No dictionary for '+key)
    return(asop_dict)

In [15]:
def compute_equalgrid_corr_global(precip,distance_bins):
    longitude = precip.coord('longitude')
    nlon=len(longitude.points)
    latitude = precip.coord('latitude')
    nlat=len(latitude.points)
    time = precip.coord('time')
    ntime=len(time.points)
    nbins = len(distance_bins)-1
    dist_centre = np.zeros(nbins)
    min_dist = np.zeros(nbins)
    max_dist = np.zeros(nbins)
    bounds = np.zeros((nbins,2))
    for b,left in enumerate(distance_bins[0:-1]):
        min_dist[b] = left
        max_dist[b] = distance_bins[b+1]
        dist_centre[b] = (max_dist[b]+min_dist[b])/2.0
        bounds[b,:] = np.asarray((min_dist[b],max_dist[b]))
    distance = iris.coords.DimCoord(dist_centre,var_name='distance',bounds=bounds)
    distance_corrs = iris.cube.Cube(np.zeros((nbins,nlat,nlon)),var_name='distance_correlations',dim_coords_and_dims=[(distance,0),(latitude,1),(longitude,2)])
    for y,latpt in enumerate(latitude.points):
        dask_distcorr=[]
        for x,lonpt in enumerate(longitude.points):
            for b in range(nbins):
                precip_mask = extract_mask_region(precip,latpt,lonpt,min_dist[b],max_dist[b])
                this_distcorr = dask.delayed(compute_gridcorr_grid)(precip[:,y,x],precip_mask)
                dask_distcorr.append(this_distcorr)
        result = dask.compute(*dask_distcorr)
        result = np.ma.asarray(result)
        result = np.reshape(result,(nbins,nlon))
        distance_corrs.data[:,y,:] = result
    return(distance_corrs)


In [16]:
def compute_spatial_summary(precip,ndivs):
    import numpy.ma as ma

    # Compute temporal summary metric only
    if not 'month_number' in [coord.name() for coord in precip.coords()]:
        iris.coord_categorisation.add_month_number(precip,'time')
    lon_coord = precip.coord('longitude')
    lat_coord = precip.coord('latitude')
    nlon = len(lon_coord.points)
    nlat = len(lat_coord.points)

    months = sorted(set(precip.coord('month_number').points))
    month_coord = iris.coords.DimCoord(months,var_name='month_number')
    nmonths = len(months)
    
    lower_thresh = iris.cube.Cube(data=np.ma.zeros((nmonths,nlat,nlon)),dim_coords_and_dims=[(month_coord,0),(lat_coord,1),(lon_coord,2)])
    lower_thresh.var_name='lower_threshold'
    lower_thresh.long_name='Lower (off) threshold based on '+str(ndivs)+' divisions'
    upper_thresh = new_cube_copy(lower_thresh,'upper_threshold','Upper (on) threshold based on '+str(ndivs)+' divisions')
    space_inter = new_cube_copy(lower_thresh,'spatial_onoff_metric','Spatial intermittency on-off metric based on '+str(ndivs)+' divisions')
    onon_freq = new_cube_copy(lower_thresh,'prob_onon','Probability of upper division neighbouring upper division')
    onoff_freq = new_cube_copy(lower_thresh,'prob_onoff','Probability of upper division neighbouring lower division')
    offon_freq = new_cube_copy(lower_thresh,'prob_offon','Probability of lower division neighbouring upper division')
    offoff_freq = new_cube_copy(lower_thresh,'prob_offoff','Probability of lower division neighbouring lower division')

    for m,month in enumerate(months):
        print('-->-->--> Month '+str(month))
        month_summaries=[]
        month_constraint = iris.Constraint(month_number=month)
        this_month = precip.extract(month_constraint)
        lower_thresh.data[m,:,:] = this_month.collapsed('time',iris.analysis.PERCENTILE,percent=100.0/ndivs).data
        upper_thresh.data[m,:,:] = this_month.collapsed('time',iris.analysis.PERCENTILE,percent=100.0*(1.0-1.0/ndivs)).data
        years = set(this_month.coord('year').points)
        nyears = len(years)
        for year in years:
            this_monthyear = dask.delayed(this_month.extract(iris.Constraint(year=year)))
            monthyear_summary = dask.delayed(compute_spatial_onoff_metric_grid)(this_monthyear,lower_thresh[m,:,:],upper_thresh[m,:,:])
            month_summaries.append(monthyear_summary)
        result = dask.compute(*month_summaries)
        result = np.ma.asarray(result)
        print(np.shape(result))

        onon_freq.data[m,:,:] = np.nanmean(result[:,0,:,:],axis=0)
        onoff_freq.data[m,:,:] = np.nanmean(result[:,1,:,:],axis=0)
        offon_freq.data[m,:,:] = np.nanmean(result[:,2,:,:],axis=0)
        offoff_freq.data[m,:,:] = np.nanmean(result[:,3,:,:],axis=0)
    
    onon_freq.data.mask = upper_thresh.data.mask
    offon_freq.data.mask = upper_thresh.data.mask
    onoff_freq.data.mask = upper_thresh.data.mask
    offoff_freq.data.mask = upper_thresh.data.mask

In [17]:
def compute_spatial_onoff_metric_grid(precip,lower_thresh,upper_thresh):
    upper_mask = precip.copy(data=np.where(precip.data >= upper_thresh.data,1,0))
    lower_mask = precip.copy(data=np.where(precip.data <= lower_thresh.data,1,0))
    non = upper_mask.collapsed('time',iris.analysis.SUM)
    noff = lower_mask.collapsed('time',iris.analysis.SUM)
    
    upper_roll = roll_count_latlon(upper_mask,3)
    onon_count = upper_roll.collapsed('time',iris.analysis.SUM) / non
    return(onon_count)

In [18]:
def roll_count_latlon(cube,window):
    roll_lat = cube.rolling_window('latitude',iris.analysis.SUM,window)
    roll_lon = roll_lat.rolling_window('longitude',iris.analysis.SUM,window)
    return(roll_lon)

In [19]:
def extract_mask_region(precip,centre_lat,centre_lon,dist_min,dist_max):
    longitude = precip.coord('longitude')
    nlon=len(longitude.points)
    latitude = precip.coord('latitude')
    nlat=len(latitude.points)
    time = precip.coord('time')
    ntime=len(time.points)
    pt_dist = np.ones((nlat,nlon))
    dN = 0 ; dS = 0 ; dlon = 0 
    for yy,target_lat in enumerate(latitude.points):
        for xx,target_lon in enumerate(longitude.points):
            pt_dist[yy,xx] = haversine((centre_lat,centre_lon),(target_lat,target_lon))
            if pt_dist[yy,xx] <= dist_max:
                dN = np.amax([dN,target_lat-centre_lat])
                dS = np.amin([dS,target_lat-centre_lat])
                if np.abs(target_lon-centre_lon) >= 180:
                    dlon = np.amax([np.abs(dlon),np.abs(np.abs(target_lon-centre_lon)-360)])
                else:
                    dlon = np.amax([dlon,np.abs(target_lon-centre_lon)])
#    if lonmin <= c
    minlon = centre_lon-dlon
    #if minlon < 0:
    #    minlon = minlon+360
    #print(centre_lon,centre_lat,dN,dS,dlon,minlon,centre_lon+dlon)
    subset = precip.intersection(longitude = (minlon,centre_lon+dlon),latitude=(centre_lat+dS,centre_lat+dN))
    dist_mask = np.ones_like(subset.data)
    for yy,target_lat in enumerate(subset.coord('latitude').points):
        for xx,target_lon in enumerate(subset.coord('longitude').points):
            pt_dist = haversine((centre_lat,centre_lon),(target_lat,target_lon))
            if pt_dist >= dist_min and pt_dist <= dist_max:
                dist_mask.data[:,yy,xx] = 0
    subset_mask = subset.copy(data=np.ma.array(subset.data,mask=dist_mask))
    return(subset_mask)

In [20]:
def compute_gridcorr_grid(precip,grid):
    import iris.analysis.stats as istats
    corr_map = istats.pearsonr(precip,grid,corr_coords='time')
    weights = iris.analysis.cartography.area_weights(corr_map)
    output=corr_map.collapsed(['longitude','latitude'],iris.analysis.MEAN,weights=weights)
    return(output.data)

In [21]:
client = Client()
dataset='GPM_IMERG'
asop_dict = get_asop_dict(dataset)

In [22]:
precip = load_cmip6(asop_dict)
masked_precip = mask_wet_season(precip)
masked_precip = mask_min_precip(masked_precip)

distributed.utils - ERROR - 'start'
Traceback (most recent call last):
  File "/home/nick/miniconda3/lib/python3.7/site-packages/distributed/utils.py", line 664, in log_errors
    yield
  File "/home/nick/miniconda3/lib/python3.7/site-packages/distributed/dashboard/components/shared.py", line 312, in update
    ts = metadata["keys"][self.key]
KeyError: 'start'
tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <zmq.eventloop.ioloop.ZMQIOLoop object at 0x7f71d5e52d50>>, <Task finished coro=<_needs_document_lock.<locals>._needs_document_lock_wrapper() done, defined at /home/nick/miniconda3/lib/python3.7/site-packages/bokeh/server/session.py:51> exception=KeyError('start')>)
Traceback (most recent call last):
  File "/home/nick/miniconda3/lib/python3.7/site-packages/tornado/ioloop.py", line 743, in _run_callback
    ret = callback()
  File "/home/nick/miniconda3/lib/python3.7/site-packages/tornado/ioloop.py", line 767, in _

In [23]:
test = compute_spatial_summary(masked_precip,4)

-->-->--> Month 1


NotYetImplementedError: Rolling window over a circular coordinate.

In [24]:
test = compute_equalgrid_corr_global(masked_precip,[0,450,900,1350])

KeyboardInterrupt: 

In [22]:
#corr_map,lag_vs_distance,autocorr,npts_map,npts = asop.compute_equalgrid_corr(masked_precip,asop_dict)

NameError: name 'asop' is not defined